In [1]:
import gzip
import requests
import random
import numpy as np
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


r/news:

In [ ]:
sub = 'news'
size = 500
after = [1615766400, # March 15, 2021 12:00:00 AM GMT
         1611964800, # January 30, 2021 12:00:00 AM GMT
         1626566400, # July 18, 2021 12:00:00 AM GMT
         1609891200, # January 6, 2021 12:00:00 AM GMT
         1616198400, # March 20, 2021 12:00:00 AM GMT
         1637452800, # November 21, 2021 12:00:00 AM GMT
         1634774400, # October 21, 2021 12:00:00 AM GMT
         1640390400, # December 25, 2021 12:00:00 AM GMT
         1627516800, # July 29, 2021 12:00:00 AM GMT
         1628380800, # August 8, 2021 12:00:00 AM GMT
         1626220800, # July 14, 2021 12:00:00 AM GMT
         1610582400, # January 14, 2021 12:00:00 AM GMT
         1613779200, # February 20, 2021 12:00:00 AM GMT
         1618704000, # April 18, 2021 12:00:00 AM GMT
         1636156800 # November 6, 2021 12:00:00 AM GMT
         ]

In [ ]:
rnews = pd.DataFrame(columns=['body'])

for epoch in after:
    url = f'https://api.pushshift.io/reddit/search/comment/?fields=body&subreddit={sub}&size={size}&after={epoch}'
    rnews = pd.concat([rnews, pd.DataFrame.from_dict(requests.get(url).json()['data'])], ignore_index=True)

In [ ]:
rnews

In [ ]:
rnews_notdel = rnews.query('body not in ["[removed]", "[deleted]"]').reset_index().drop(columns=['index'])
rnews_notdel

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='rnews_data.csv')
rnews_notdel.to_csv('rnews_data.zip', index=False, compression=compression_opts)

In [ ]:
rnews = pd.read_csv('rnews_data.csv')
rnews

In [ ]:
random.seed(1)
n = rnews.shape[0]
indices = random.sample(range(n), 30)
rnews.iloc[indices].to_csv('rnews_sample30.txt', index=False)

Amazon book reviews:

In [ ]:
with gzip.open('Kindle_Store_5.json.gz', 'rb') as f:
    reviews_df = pd.read_json(f, orient='records', lines=True)

In [ ]:
random.seed(1)
reviews_data = reviews_df[['reviewText']].iloc[random.sample(range(reviews_df.shape[0]), 1000)]
reviews_data

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='reviews_data.csv')
reviews_data.to_csv('reviews_data.zip', index=False, compression=compression_opts)

In [ ]:
random.seed(1)
reviews_data.reset_index().drop(
        columns=['index']
    ).iloc[random.sample(range(1000), 20)].to_csv(
        'reviews_sample30.csv', index=False
    )

Annotation (AP2):

In [ ]:
import zipfile

with zipfile.ZipFile('data/reviews_data.zip', 'r') as z:
    with z.open('reviews_data.csv') as f:
        reviews_data = pd.read_csv(f)

In [ ]:
reviews_data

In [14]:
reviews_eval = pd.read_csv('data/reviews_eval_adj.csv')
reviews_exp = pd.read_csv('data/reviews_exp_adj.csv').convert_dtypes()

In [15]:
display(reviews_eval.head())
display(reviews_exp.head())

,text,stalebi,zhossainzadeh,adjudicated
0,"Up until reading Hunger Games, this was my all...",4,3,3
1,I read this book before watching the movie (th...,4,3,3
2,"In a way I never expected it, this topped the ...",4,3,3
3,This is an excellent story. It picks up the Ou...,4,3,4
4,"Like Ismae, the protagonist of Grave Mercy, Sy...",4,3,4


,text,stalebi,zhossainzadeh,adjudicated
0,One of the most enjoyable stories I've read in...,4,4,4
1,A hilarious and book from Tina Fey - definitel...,4,3,4
2,Highly recommended if you are curious about Ma...,4,3,3
3,This story had so much promise! And I love Whi...,4,3,4
4,"Another warm, romantic and totally hot story! ...",4,3,4


In [17]:
reviews_annotated = pd.concat([reviews_eval, reviews_exp], ignore_index=True)

In [50]:
reviews_annotated.value_counts('adjudicated')

adjudicated
1    403
2    328
3    187
4     86
dtype: int64

In [47]:
adjudicated = reviews_annotated[['text', 'adjudicated']].melt(
        id_vars=['text'], 
        value_vars=['adjudicated'], 
        var_name='adjudicated'
    ).rename(
        columns={'value': 'label'}
    )[['adjudicated', 'label', 'text']]

In [49]:
adjudicated.to_csv('data/adjudicated.txt', sep='\t', header=False)

In [71]:
individual_annotations = reviews_eval[['text', 'stalebi', 'zhossainzadeh']].reset_index().melt(
        id_vars=['index', 'text'],
        value_vars=['stalebi', 'zhossainzadeh'],
        var_name='annotator'
    ).sort_values(
        'index', 
        ignore_index=True
    ).rename(
        columns={'value': 'label'}
    )[['index', 'annotator', 'label', 'text']]

In [72]:
individual_annotations

,index,annotator,label,text
0,0,stalebi,4,"Up until reading Hunger Games, this was my all..."
1,0,zhossainzadeh,3,"Up until reading Hunger Games, this was my all..."
2,1,stalebi,4,I read this book before watching the movie (th...
3,1,zhossainzadeh,3,I read this book before watching the movie (th...
4,2,stalebi,4,"In a way I never expected it, this topped the ..."
...,...,...,...,...
1003,501,stalebi,1,"3.5 stars, 8th&up Enjoyable series, & I'm hook..."
1004,502,zhossainzadeh,1,"3.5 stars, 5th&up Although not as action packe..."
1005,502,stalebi,1,"3.5 stars, 5th&up Although not as action packe..."
1006,503,stalebi,1,"3.5 stars Despite a little bit of a let-down, ..."


In [74]:
individual_annotations.to_csv('data/individual_annotations.txt', sep='\t', index=False, header=False)